# Classes

Comme on l'a vu, en python, tout est objet.

Les classes sont incontournables. Elles permettent de structure "proprement" un projet, en encapsulant des valeurs et des fonctionnalités au sein d'une même entité.

Exemple

In [ ]:
class Counter:
    
    def reset(self,init=0):
        self.count = init
        
    def add(self):
        self.count += 1
        return self.count
    
counter = Counter()
counter.reset(0) 
print(counter.add())
print(counter.add())
print(counter.add())

Commentaires :
- Mot clef `class`
- Les methodes sont des fonctions dont le premier argument est `self`
- `self` represente l'objet, instance de la classe
- `self` a des attributs, accessibles par l'opérateur `.` ils peuvent être ajoutés et modifiés **sans contraintes**
- Une classe est instanciée par son appel (`counter = Counter()`)

Les attributs peuvent être aussi modifiés et ajoutés depuis l'objet lui même !

In [ ]:
obj1 = Counter()
obj1.x = 5
x = 7
print("x in object 1 =",obj1.x)

### Initialisation

In [ ]:
class Counter:
    
    def __init__(self, step=1):
        self.step = step
        self.count = 0
    
    def reset(self,init=0):
        self.count = init
        
    def add(self):
        self.count += self.step
        return self.count
    

`__init__`, qui se prononce "dunder init" est apellé au moment de l'instanciation. 

In [ ]:
c = Counter(step=2)
print(c.add())
print(c.add())
print(c.add())

**dir()** est utile pour connaître les attributs et méthodes accessibles

In [ ]:
print("Contents of Counter class:",dir(Counter) )
print("------------------------")
print("------------------------")
print("Contents of counter object:", dir(c))

## `__dict__`

`__dict__` est le dictionnaire des attributs de l'object. 

On peut le consulter ou le manipuler directement.

In [ ]:
print(c.__dict__)
c.newattr = "newval"
print(c.__dict__)
print(c.__dict__["newattr"])
c.__dict__["newattr"] = "another val"
print(c.newattr)
print(c.__dict__)

On voit ici que les attributs d'un object sont implémentés comme un dictionnaire python. Cela a des coûts en taille et en performance. 

Si l'ensemble des attributs possible est fixé d'avance, il est possible d'empecher python d'utiliser un dictionnaire, par l'utilisation de `__slots__`.

In [ ]:
class Counter:
    __slots__= ("step", "count")
    def __init__(self, step=1):
        self.step = step
        self.count = 0
    
    def reset(self,init=0):
        self.count = init
        
    def add(self):
        self.count += self.step
        return self.count

In [ ]:
# Counter().__dict__   # AttributeError : 'Counter' object has no attribute '__dict__'
# Counter().newattr = "newval"  # AttributeError: 'Counter' object has no attribute 'newattr'

Plus d'informations [ici](https://stackoverflow.com/questions/472000/usage-of-slots)

Nous reconnaissons les attributs et méthodes que nous avons défini (add, count, reset, step, `__init__`) mais aussi un certain nombres de méthodes et attributs herités du type `Object`. 

Ces méthodes sont utilisés selon le contexte d'utilisation de l'objet. Quelques exemples :

- `__hash__()` invoqué en interne pour obtenir un hash de l'objet (utile dans le cadre de sets ou de dictionnaires)
- `__getattribute__()` invoqué lorsqu'on recherche un attribut.
- `__str__()` invoqué lors de str(monobjet) ou print(monobjet)
- `__repr__()` utilisé à la place de `__str__()` si ce dernier n'est pas défini. Vise à fournir sous forme de texte une représentation plus précise que `__str__()`.

Il est possible, et parfois utile, de les redéfinir soi même (`__init__` par exemple).

Beaucoup d'autres dunders methodes (autre que celles déjà présentes) peuvent être définis pour augmenter les fonctionnalités de la classe :

- `__get_attr__` invoqué lorsqu'un attribut est introuvable
- `__iter__` invoqué lorsqu'on veut itérer sur votre object (comme dans le cadre d'une boucle for)
- `__len__` invoqué lors de len(monobjet)


Les dunders ne doivent pas être considérés comme des attributs ou méthodes privées. D'ailleurs le concept n'existe pas en python. Mais plutôt comme des méthodes utiles (on les apelle d'ailleurs parfois "magic methods").


Le sujet des dunder methods est riche et vaste. Liens pour en savoir plus :
- [Documentation](https://docs.python.org/3/reference/datamodel.html) . 
- [Excellente vidéo](https://www.youtube.com/watch?v=M4gPxbo6G6k) de l'auteur du livre du livre Fluent Python, portant notamment sur l'utilité des dunders.

## Heritage

Python suporte l'héritage

In [8]:
class Person:
    
    def __init__(self, name):
        self.name = name
    
    def say_hello(self):
        print("hello")

In [9]:
class Developper(Person):
    
    def code(self):
        print("I code")

In [11]:
tom = Developper("tom")
tom.code()
tom.say_hello()
print(tom.name)

I code
hello
tom


## Heritage multiple

L'héritage multiple est possible

In [12]:
class YiellingAbility:
    def yell(self):
        print("AHHHHHHHHH")

class Developper(Person, YiellingAbility):
    def code(self):
        print("I code")

In [13]:
tom = Developper("tom")
tom.yell()

AHHHHHHHHH
